In [1]:
import pandas as pd
import xgboost as xgb

C:\Anaconda3\lib\site-packages\sklearn\cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train=pd.read_hdf('train.h5')

In [3]:
X=train.iloc[:,:-1]
Y=train.iloc[:,-1:]

In [4]:
from sklearn.model_selection import train_test_split
#xgbmodel needs optimize
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.5, random_state=42)
import xgboost as xgb
dtrain = xgb.DMatrix(X_train.as_matrix(),label=y_train.as_matrix())
dtest = xgb.DMatrix(X_test.as_matrix(),label=y_test.as_matrix())
watchlist  = [(dtrain,'train'),(dtest,'test')]




In [21]:
param = {'max_depth':6, 
             'eta':0.1, 
             'silent':False, 
             'objective':'reg:linear', 
             'verbose_eval':True, 
             'eval_metric' : 'rmse', 
             'colsample_bytree': 0.7,
             'subsample': 0.7,
             'nthread': 4,
             'min_child_weight': 1,
             'lambda':0.1
            }
num_round = 250
#bst = xgb.train(param, dtrain, num_round, watchlist,early_stopping_rounds=10)
#pred=bst.predict(dtest)
xgb.cv(param, dtrain, num_boost_round=100, nfold=3, stratified=False, folds=None, 
           early_stopping_rounds=10, fpreproc=None, as_pandas=True, verbose_eval=True, show_stdv=True, seed=0)


C:\Anaconda3\lib\site-packages\xgboost-0.4-py3.5.egg\xgboost\training.py:270: VisibleDeprecationWarning: using a non-integer number instead of an integer will result in an error in the future
  idset = [randidx[(i * kstep): min(len(randidx), (i + 1) * kstep)] for i in range(nfold)]


,test-rmse-mean,test-rmse-std,train-rmse-mean,train-rmse-std
0,0.450365,0.000039,0.450365,0.000014
1,0.405447,0.000041,0.405447,0.000011
2,0.365031,0.000041,0.365031,0.000011
3,0.328674,0.000041,0.328673,0.000011
4,0.295968,0.000043,0.295966,0.000008
5,0.266549,0.000043,0.266546,0.000008
6,0.240092,0.000043,0.240087,0.000008
7,0.216302,0.000042,0.216297,0.000007
8,0.194918,0.000041,0.194911,0.000008
9,0.175697,0.000040,0.175689,0.000008


In [9]:
import numpy as np
import math
def score(y, pred_y):
    R2 = 1 - np.sum((y - pred_y)**2) / np.sum((y - np.mean(y))**2)
    R = np.sign(R2) * math.sqrt(abs(R2))
    return(R)

In [25]:
from matplotlib import pyplot
xgb.plot_importance(bst)
pyplot.show()

In [45]:
colnames=train.columns.tolist()
colnames=colnames[:-1]

In [80]:
x=[]
y=[]
fscore=bst.get_fscore()
for i in range(len(colnames)):
    f='f'+str(i)
    x.append(colnames[i])
    y.append(fscore[f])
data=pd.DataFrame({'Variable':x,'Score':y})  
data.sort_values(['Score'],inplace=True,ascending=False)
    

In [85]:
top_10=data.head(20)
pyplot.bar(range(len(top_10)),top_10['Score'])
pyplot.xticks(range(len(top_10)),top_10['Variable'],rotation='vertical')
pyplot.show()

In [78]:
data.sort_values(['Score'],inplace=True,ascending=False)

In [84]:
data[data['Variable']=='id']

,Score,Variable
0,39,id


In [ ]:
xgb_model = xgb.XGBRegressor()

#when in doubt, use xgboost
parameters = {'nthread':[4], #when use hyperthread, xgboost may become slower
              'objective':['reg:linear'],
              'learning_rate': [0.15,1], #so called `eta` value
              'max_depth': [8],
              'min_child_weight': [1],
              'silent': [0],
              'subsample': [0.9],
              'colsample_bytree': [0.5],
              'seed': [1337]}


#should evaluate by train_eval instead of the full dataset
clf = GridSearchCV(xgb_model, parameters, n_jobs=4, 
                   cv=3, 
                    scoring='r2',
                   verbose=2, refit=True)

clf.fit(X_train,y_train)

Fitting 3 folds for each of 2 candidates, totalling 6 fits


In [38]:
y_train

,y
1284628,0.053411
493739,0.019886
1429998,0.031907
1109145,-0.021328
1371185,0.014981
695710,0.007919
1437774,-0.050029
1113093,-0.005094
279830,0.013330
318554,0.013596


In [ ]:
x_test=X[0:10000]
y_test=Y[0:10000]
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
iris = datasets.load_iris()
param = {'max_depth':[4,3], 
             'learning_rate':[1,0.5,0.1], 
             'silent':[False], 
             'objective':['reg:linear'], 
             'colsample_bytree': [0.95],
             'subsample': [0.95],
             'nthread': [4],
             'min_child_weight': [1],
             'n_estimators':[100,200]
}
             

xgb = xgb.XGBRegressor()
clf = GridSearchCV(xgb,  param,scoring='r2',cv=3)
clf.fit(x_test, y_test)

In [ ]:
a=pd.DataFrame(clf.cv_results_)
a

In [55]:
a=pd.DataFrame(clf.cv_results_)


In [56]:
a

,mean_fit_time,mean_score_time,mean_test_score,mean_train_score,param_colsample_bylevel,param_colsample_bytree,param_gamma,param_learning_rate,param_max_depth,param_min_child_weight,...,split0_test_score,split0_train_score,split1_test_score,split1_train_score,split2_test_score,split2_train_score,std_fit_time,std_score_time,std_test_score,std_train_score
0,3.059707,0.042361,-1.093638,0.998012,1,1,0,1,8,1,...,-1.816083,0.997114,-0.769061,0.998734,-0.695553,0.998188,0.065436,0.036205,0.511803,0.000673
1,2.448967,0.015344,-1.076655,0.988926,1,1,0,1,7,1,...,-1.893898,0.984238,-0.625746,0.991071,-0.710076,0.991469,0.089638,0.000624,0.578989,0.003319
2,2.106405,0.014676,-1.005028,0.962186,1,1,0,1,6,1,...,-1.829801,0.954071,-0.626897,0.966363,-0.558139,0.966125,0.088604,0.000236,0.583965,0.005739
3,1.787192,0.012842,-0.953997,0.900108,1,1,0,1,5,1,...,-1.708733,0.893661,-0.621154,0.901152,-0.531877,0.905510,0.108406,0.000624,0.535002,0.004894
4,1.460808,0.013009,-0.734522,0.793140,1,1,0,1,4,1,...,-1.366215,0.778815,-0.397064,0.802341,-0.440097,0.798264,0.090944,0.000707,0.447087,0.010265
5,1.114410,0.012675,-0.605328,0.629850,1,1,0,1,3,1,...,-1.289973,0.624608,-0.243107,0.642125,-0.282698,0.622818,0.043550,0.000850,0.484460,0.008710
6,0.762010,0.010340,-0.402388,0.414559,1,1,0,1,2,1,...,-0.844019,0.413553,-0.110889,0.408309,-0.252123,0.421815,0.010471,0.000472,0.317604,0.005560
7,2.927119,0.017011,-0.660511,0.978301,1,1,0,0.5,8,1,...,-1.238484,0.972284,-0.332382,0.976013,-0.410492,0.986605,0.155372,0.001473,0.409992,0.006066
8,3.198804,0.019013,-0.610722,0.947102,1,1,0,0.5,7,1,...,-1.228188,0.946083,-0.266991,0.948690,-0.336802,0.946534,0.353681,0.005311,0.437609,0.001138
9,2.269514,0.014510,-0.588892,0.893957,1,1,0,0.5,6,1,...,-1.212182,0.885048,-0.198200,0.891714,-0.356106,0.905108,0.176720,0.000408,0.445487,0.008341


In [66]:
X.shape

(1710756, 110)

In [9]:
a=train[train.id==11]

In [20]:
from sklearn.model_selection import train_test_split
#xgbmodel needs optimize
X_train, X_test, y_train, y_test = train_test_split(a.iloc[:,:-1],a.iloc[:,-1:], test_size=0.5, random_state=42)
import xgboost as xgb
dtrain = xgb.DMatrix(X_train.as_matrix(),label=y_train.as_matrix())
dtest = xgb.DMatrix(X_test.as_matrix(),label=y_test.as_matrix())
watchlist  = [(dtrain,'train'),(dtest,'test')]

In [21]:
param = {'max_depth':6, 
             'eta':0.1, 
             'silent':False, 
             'objective':'reg:linear', 
             'verbose_eval':True, 
             'eval_metric' : 'rmse', 
             'colsample_bytree': 0.7,
             'subsample': 0.7,
             'nthread': 4,
             'min_child_weight': 1,
             'lambda':0.1
            }
num_round = 250
bst = xgb.train(param, dtrain, num_round, watchlist,early_stopping_rounds=10)
#pred=bst.predict(dtest)
#xgb.cv(param, dtrain, num_boost_round=100, nfold=3, stratified=False, folds=None, 
#           early_stopping_rounds=10, fpreproc=None, as_pandas=True, verbose_eval=True, show_stdv=True, seed=0)


Will train until test error hasn't decreased in 10 rounds.
[0]	train-rmse:0.450076	test-rmse:0.449918
[1]	train-rmse:0.405118	test-rmse:0.404965
[2]	train-rmse:0.364642	test-rmse:0.364494
[3]	train-rmse:0.328237	test-rmse:0.328095
[4]	train-rmse:0.295432	test-rmse:0.295297
[5]	train-rmse:0.265948	test-rmse:0.265820
[6]	train-rmse:0.239428	test-rmse:0.239309
[7]	train-rmse:0.215549	test-rmse:0.215438
[8]	train-rmse:0.194085	test-rmse:0.193997
[9]	train-rmse:0.174765	test-rmse:0.174684
[10]	train-rmse:0.157367	test-rmse:0.157308
[11]	train-rmse:0.141733	test-rmse:0.141714
[12]	train-rmse:0.127661	test-rmse:0.127658
[13]	train-rmse:0.115046	test-rmse:0.115089
[14]	train-rmse:0.103649	test-rmse:0.103706
[15]	train-rmse:0.093418	test-rmse:0.093505
[16]	train-rmse:0.084215	test-rmse:0.084326
[17]	train-rmse:0.075928	test-rmse:0.076076
[18]	train-rmse:0.068531	test-rmse:0.068754
[19]	train-rmse:0.061841	test-rmse:0.062141
[20]	train-rmse:0.055841	test-rmse:0.056227
[21]	train-rmse:0.050472	te

In [22]:
pred=bst.predict(dtest)

In [39]:
dtest.

array([  5.61443856e-03,  -3.89450882e-03,  -4.88554128e-04,
         1.20551949e-02,  -8.24266672e-03,   1.77409369e-02,
        -2.44538509e-03,   3.68521432e-03,  -6.49402477e-03,
        -5.73176984e-03,   9.30446142e-04,  -1.54063003e-02,
        -4.73981397e-03,  -2.97688274e-03,  -4.28709434e-03,
         6.67398213e-04,  -2.02798052e-03,   7.45899230e-03,
        -1.25573529e-02,  -3.02784750e-03,  -1.13810115e-02,
        -8.66744667e-03,   1.14719169e-02,  -5.35084540e-03,
        -6.20089360e-02,  -6.04476593e-03,  -9.74843930e-03,
         6.65017683e-03,   1.36424997e-03,  -1.69146666e-03,
         3.13457241e-03,  -1.54513130e-02,   1.06652044e-02,
        -4.13901173e-03,   3.12851719e-03,  -6.94774149e-04,
         1.68138202e-02,  -5.43007115e-03,  -1.04733715e-02,
        -4.51213541e-03,   1.30075701e-02,  -9.77868214e-03,
         7.01521784e-02,   9.85227991e-03,  -1.47428794e-03,
        -1.13475486e-03,   1.82984602e-02,   1.04475506e-02,
         7.95560237e-03,

In [61]:
import numpy as np
import matplotlib.pyplot as plt
t=np.linspace(0,1600,16000)
sin=0.02*np.sin(2*t)
plt.plot(a['timestamp'],a['y'])
plt.plot(predict.x,predict.y)
plt.plot(t,sin)
plt.show()

In [54]:
t

array([ 0.])

In [41]:
predict=pd.DataFrame({'x':X_test['timestamp'],'y':pred})

In [44]:
predict.sort_values(['x'],inplace=True)

In [46]:
predict

,x,y
2251,3,-0.000080
3751,5,-0.011402
7501,10,0.004633
9001,12,0.003421
11248,15,0.002562
13493,18,-0.003716
17228,23,-0.009597
17975,24,-0.000801
21699,29,-0.011342
22443,30,-0.010603
